In [1]:
from pathlib import Path
import numpy as np
from re_alc import *

path_dataset_train = '../TP/template-alumnos/dataset/cats_and_dogs/train'
path_dataset_val = '../TP/template-alumnos/dataset/cats_and_dogs/val'

def cargarDataset(carpeta: Path):
    """
    carpeta Path a la carpeta de train / val
    retorna la matriz de embeddings (1536x2000 / 1536x1000) y la matriz Y (2x2000 / 2x1000)
    """
    
    pathCats = carpeta.joinpath('cats/efficientnet_b3_embeddings.npy')
    pathDogs = carpeta.joinpath('dogs/efficientnet_b3_embeddings.npy')

    embeddingsCats = np.load(pathCats)
    embeddingsDogs = np.load(pathDogs)

    embeddings = np.concatenate((embeddingsCats, embeddingsDogs), axis=1)
    _, m = embeddings.shape

    Y = np.zeros((2,m))
    for i in range(0, int(m/2)):
        Y[0,i] = 1

    for i in range(int(m/2), m):
        Y[1,i] = 1

    return embeddings, Y

def cargarDatasetCompleto():
    """
    retorna el dataset de entrenamiento (X_t, Y_t) y el de valuación (X_v, Y_v)
    """

    
    Xt, Yt = cargarDataset(Path(path_dataset_train))
    Xv, Yv = cargarDataset(Path(path_dataset_val))

    return Xt, Yt, Xv, Yv

Las siguientes funciones generan la matriz W a partir del calculo de la pseudoinversa de X utilizando diferentes metodos de decomposicion

In [ ]:
def generar_W_EcuacionesNormales():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    n, p = Xt.shape
    rangoX = min(n, p)

    if rangoX == p and rangoX < n:
        XtX = Xt.T@Xt
        L = calculaCholesky(XtX)
        W = pinvEcuacionesNormales(Xt, L, Yt)

    elif rangoX == n and rangoX < p:
        XXt = Xt@Xt.T
        L = calculaCholesky(XXt)
        W = pinvEcuacionesNormales(Xt, L, Yt)

    elif rangoX == p and p == n:
        W = pinvEcuacionesNormales(Xt, None, Yt)


    np.save('./W_eqnormales', W)

def generar_W_QRHH():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_HH(Xt.T)
    W = pinvHouseHolder(Q, R, Yt)
    np.save('./W_qrhh', W)

def generar_W_QRGS():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    Q, R = QR_con_GS(traspuesta(Xt)) # type: ignore
    W = pinvGramSchmidt(Q, R, Yt)
    np.save('./W_qrgs', W)

def generar_W_SVD():
    Xt, Yt = cargarDataset(Path(path_dataset_train))

    # SVD REDUCIDA ME DA V1 DIRECTAMENTE
    U, S, V1 = svd_reducida(Xt)

    W = pinvSVD(U, S, V1, Yt)
    np.save('./W_SVD', W)
    

Utilizando estas cuatro funciones, generamos la matriz de pesos *W* con los distintos métodos

In [8]:
generar_W_EcuacionesNormales()

In [ ]:
generar_W_QRGS()

In [ ]:
generar_W_QRHH()

In [ ]:
generar_W_SVD()